# **Statistics(I)(2) - Final Project Code**
## **Task 1**

In [30]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import statsmodels.stats.api as sms
from scipy import stats
import seaborn as sns
import math

In [31]:
def f_test_variances(x1, x2, sides, alpha):
    a1 = np.array(x1)
    a2 = np.array(x2)

    result = np.full((7, 2), None, dtype=float)
    
    result[0] = [np.mean(a1), np.mean(a2)]                    # Means
    result[1] = [np.std(a1, ddof=1), np.std(a2, ddof=1)]      # Sample standard deviations
    result[2] = [a1.size, a2.size]                            # Sample sizes

    dfn, dfd = a1.size - 1, a2.size - 1
    result[3] = [dfn, dfd]

    # F-statistic: ratio of sample variances
    f_stat = np.var(a1, ddof=1) / np.var(a2, ddof=1)
    result[4, 0] = f_stat

    # Critical F-values
    if sides > 1:
        result[5] = [
            stats.f.isf(alpha / 2, dfn, dfd),  # Upper critical value
            stats.f.ppf(alpha / 2, dfn, dfd)   # Lower critical value
        ]
    else:
        result[5] = [
            stats.f.isf(alpha, dfn, dfd),      # One-sided critical value
            stats.f.ppf(alpha, dfn, dfd)
        ]

    if f_stat > 1:
        p_value = 1 - stats.f.cdf(f_stat, dfn, dfd)
    else:
        p_value = stats.f.cdf(f_stat, dfn, dfd)
    if sides > 1:
        p_value *= 2
    result[6, 0] = p_value

    labels = ['Mean', 'Std Dev', 'Size', 'Degrees of Freedom', 'F-statistic', 'F-critical', 'p-value']
    df = pd.DataFrame(result, index=labels)

    return df

In [32]:
def t_test_summary(x1, x2, usevar='pooled', alpha=0.05):
    """
    Runs a t-test using statsmodels' CompareMeans and prints a summary.

    Parameters:
    - df: DataFrame containing the columns
    - col1, col2: Column names to compare
    - usevar: 'pooled' (equal variances) or 'unequal' (Welch)
    - alpha: Significance level
    """
    group1 = sms.DescrStatsW(x1)
    group2 = sms.DescrStatsW(x2)
    t_test = sms.CompareMeans(group1, group2)
    
    return t_test.summary(usevar=usevar, alpha=alpha)

In [33]:
df = pd.read_csv("movie_summary.csv")

filtered_df = df[
    (df['genre'] != "Reality") &
    (df['production_budget'] != 0) &
    (df['production_year'] > 2000) &
    df['genre'].notna() &
    df['domestic_box_office'].notna() &
    df['international_box_office'].notna()
].copy()

filtered_df["total_box_office"] = filtered_df["domestic_box_office"] + filtered_df["international_box_office"]
filtered_df["flattened_production_budget"] = np.pow(filtered_df["production_budget"], 1/8)
filtered_df["flattened_total_box_office"] = np.pow(filtered_df["total_box_office"], 1/8)

filtered_df['budget_category'] = pd.cut(
    filtered_df['production_budget'], 
    bins=[10000, 1700000, 5000000], 
    labels=['Low', 'High']
)

budget_low = filtered_df[filtered_df['budget_category'] == 'Low']['flattened_total_box_office']
budget_high = filtered_df[filtered_df['budget_category'] == 'High']['flattened_total_box_office']

In [34]:
f_test_variances(budget_low, budget_high, sides=2, alpha=0.05)

,0,1
Mean,5.249975,6.687560
Std Dev,1.845139,1.903176
Size,339.000000,411.000000
Degrees of Freedom,338.000000,410.000000
F-statistic,0.939940,NaN
F-critical,1.225001,0.814671
p-value,0.554008,NaN


In [35]:
t_test_summary(budget_low, budget_high, usevar='pooled', alpha=0.05)

,coef,std err,t,P>|t|,[0.025,0.975]
subset #1,-1.4376,0.138,-10.438,0.000,-1.708,-1.167
